In [15]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from datetime import datetime
from sklearn import preprocessing
from sklearn.cluster import *
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benxi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Read Data

In [11]:
df = pd.read_csv('../data.csv',sep='|')
df


,link,title,author,date,content
0,https://www.scmp.com/comment/opinion/article/3...,"From Israel to India, America’s illiberal alli...",Mohamed Zeeshan,2023-11-04T12:30:08.000Z,Having spent the past several months focusing ...
1,https://www.scmp.com/news/world/middle-east/ar...,Israel-Gaza war: UN human rights chief hits ou...,Agence France-Presse,2023-11-04T12:07:19.000Z,The United Nations human rights chief said on...
2,https://www.scmp.com/news/world/middle-east/ar...,Israeli air strikes on Gaza continue overnight...,Agence France-Presse,2023-11-04T09:45:11.000Z,Fighting raged in Gaza on Saturday for a 29th ...
3,https://www.scmp.com/news/world/middle-east/ar...,Israel’s Netanyahu resists US pressure to paus...,Associated Press,2023-11-04T03:28:21.000Z,Israeli Prime Minister Benjamin Netanyahu on F...
4,https://www.scmp.com/news/world/middle-east/ar...,"Israel strikes ambulance in Gaza City, many re...",Reuters,2023-11-03T20:03:59.000Z,Israel struck an ambulance in Gaza City on Fri...
...,...,...,...,...,...
252,https://www.scmp.com/news/world/united-states-...,Israel-Gaza war: Donald Trump slammed for call...,Reuters,2023-10-12T18:06:01.000Z,Israel and the White House on Thursday condemn...
253,https://www.scmp.com/news/hong-kong/politics/a...,Hong Kong Muslim groups register protest with ...,Lo Hoi-ying,2023-10-12T14:20:33.000Z,Muslim groups in Hong Kong have registered a s...
254,https://www.scmp.com/news/china/diplomacy/arti...,Israel-Hamas war: mother of kidnapped woman sa...,Sylvie Zhuang,2023-10-12T12:46:56.000Z,"The mother of Noa Argamani, a woman believed t..."
255,https://www.scmp.com/comment/opinion/article/3...,Israel-Hamas war shows limits of forecasting i...,Nicholas Spiro,2023-10-12T12:30:11.000Z,Boxer Mike Tyson’s famous quote – “Everyone ha...


In [12]:
contents= df['content'].copy()
dates = df['date'].copy()

# contents.head(3)
# contentList=[]
# for text in contents:
#     tokens = word_tokenize(text)
#     for index,token in enumerate(tokens):
#         tokens[index]=token.lower()
#     contentList.append(tokens)
    

# print(contentList[0])

# normalize
for index,timeString in enumerate(dates):
    timeStr = datetime.strptime(timeString, '%Y-%m-%dT%H:%M:%S.%fZ')
    formatted_time = timeStr.strftime('%Y%m%d%H%M%S')
    dates[index]=int(formatted_time)
print(dates[0:3])

normalied_dates=(dates-dates.min())/(dates.max()-dates.min())
normalied_dates:np.ndarray = normalied_dates.values.reshape(-1,1) #convert to numpy array
print(normalied_dates[0:3])


0    20231104123008
1    20231104120719
2    20231104094511
Name: date, dtype: object
[[1.0]
 [0.9999751220400336]
 [0.9996902808103263]]


In [13]:
tf_idf=TfidfVectorizer(stop_words=stopwords.words('english'),lowercase=True) # other possible method doc2vec
x_tf_idf = tf_idf.fit_transform(contents)
x_tf_idf=x_tf_idf.todense()
print(x_tf_idf.shape)

(257, 5653)


In [14]:
# merge the feature
X_train = np.hstack((x_tf_idf,normalied_dates))
print(X_train.shape)

(257, 5654)


In [47]:
# clustering
clustering = MeanShift(bandwidth=1.2643,n_jobs=-1).fit(np.asarray(X_train)) # other possbile clustering dbscan, bayes gaussian mixture
print(len(clustering.cluster_centers_))

7


In [49]:
#label text
group=clustering.predict(np.asarray(X_train))
print(group.shape)
df['group']=group
df

(257,)


,link,title,author,date,content,group
0,https://www.scmp.com/comment/opinion/article/3...,"From Israel to India, America’s illiberal alli...",Mohamed Zeeshan,2023-11-04T12:30:08.000Z,Having spent the past several months focusing ...,4
1,https://www.scmp.com/news/world/middle-east/ar...,Israel-Gaza war: UN human rights chief hits ou...,Agence France-Presse,2023-11-04T12:07:19.000Z,The United Nations human rights chief said on...,0
2,https://www.scmp.com/news/world/middle-east/ar...,Israeli air strikes on Gaza continue overnight...,Agence France-Presse,2023-11-04T09:45:11.000Z,Fighting raged in Gaza on Saturday for a 29th ...,0
3,https://www.scmp.com/news/world/middle-east/ar...,Israel’s Netanyahu resists US pressure to paus...,Associated Press,2023-11-04T03:28:21.000Z,Israeli Prime Minister Benjamin Netanyahu on F...,0
4,https://www.scmp.com/news/world/middle-east/ar...,"Israel strikes ambulance in Gaza City, many re...",Reuters,2023-11-03T20:03:59.000Z,Israel struck an ambulance in Gaza City on Fri...,0
...,...,...,...,...,...,...
252,https://www.scmp.com/news/world/united-states-...,Israel-Gaza war: Donald Trump slammed for call...,Reuters,2023-10-12T18:06:01.000Z,Israel and the White House on Thursday condemn...,0
253,https://www.scmp.com/news/hong-kong/politics/a...,Hong Kong Muslim groups register protest with ...,Lo Hoi-ying,2023-10-12T14:20:33.000Z,Muslim groups in Hong Kong have registered a s...,0
254,https://www.scmp.com/news/china/diplomacy/arti...,Israel-Hamas war: mother of kidnapped woman sa...,Sylvie Zhuang,2023-10-12T12:46:56.000Z,"The mother of Noa Argamani, a woman believed t...",0
255,https://www.scmp.com/comment/opinion/article/3...,Israel-Hamas war shows limits of forecasting i...,Nicholas Spiro,2023-10-12T12:30:11.000Z,Boxer Mike Tyson’s famous quote – “Everyone ha...,0


In [80]:
#pick earliest news
groupDates=np.hstack((dates.values.reshape(-1,1),group.reshape(-1,1)))
groupDates=pd.DataFrame(groupDates,columns=["date","group"])
groupDates=groupDates.astype({"date":'Int64',"group":'int'})
result = groupDates.groupby('group')['date'].idxmin()
print(groupDates)
print(result)


               date  group
0    20231104123008      4
1    20231104120719      0
2    20231104094511      0
3    20231104032821      0
4    20231103200359      0
..              ...    ...
252  20231012180601      0
253  20231012142033      0
254  20231012124656      0
255  20231012123011      0
256  20231012113856      0

[257 rows x 2 columns]
group
0    256
1      9
2     11
3     14
4      0
5      5
6     24
Name: date, dtype: int64


In [83]:
# selected article
df.loc[result.values]

,link,title,author,date,content,group
256,https://www.scmp.com/news/world/middle-east/ar...,Israel-Gaza war: Blinken tells Netanyahu US ‘b...,Reuters,2023-10-12T11:38:56.000Z,US Secretary of State Antony Blinken on Thursd...,0
9,https://www.scmp.com/comment/opinion/article/3...,Ukraine and Gaza wars have put the West in the...,Andrew Sheng,2023-11-03T12:30:08.000Z,The images and news coming out of Gaza are so...,1
11,https://www.scmp.com/sport/football/article/32...,2026 Fifa World Cup: Hong Kong defender Li Nga...,Paul McNamara,2023-11-03T07:30:23.000Z,Hong Kong defender Li Ngai-hoi has batted away...,2
14,https://www.scmp.com/news/world/americas/artic...,UN votes overwhelmingly to end US embargo on C...,Reuters,2023-11-03T03:11:24.000Z,The UN General Assembly called for the 31st ti...,3
0,https://www.scmp.com/comment/opinion/article/3...,"From Israel to India, America’s illiberal alli...",Mohamed Zeeshan,2023-11-04T12:30:08.000Z,Having spent the past several months focusing ...,4
5,https://www.scmp.com/news/world/middle-east/ar...,"‘No income, no life’: Jerusalem’s Old City suf...",Agence France-Presse,2023-11-03T15:45:14.000Z,The once bustling streets of Jerusalem’s Old C...,5
24,https://www.scmp.com/comment/opinion/article/3...,"Israel’s borders are safe, but Gaza war puts i...",Richard Harris,2023-11-02T05:30:17.000Z,One of the world’s largest industries in terms...,6
